# Moving average trading

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from datetime import datetime
import security_comparison as sc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import HBox, VBox, TwoByTwoLayout, Label
from ipywidgets import interactive, interactive_output
from ipywidgets import FloatSlider, IntSlider, SelectionSlider, SelectionRangeSlider
from IPython.display import display

%matplotlib inline
%load_ext autoreload
%autoreload 2

### Default parameters

In [2]:
DIRNAME  = 'data'
PLOT_DIR = 'plots'
HUGE     = 1000000
DEFAULT_PERIOD = '5y'
FEE_PCT        = .004  # broker's fee
initial_wealth = 100.0 
pd.options.display.float_format = '{:,.2f}'.format

# ---------------------------#
TICKER     = 'AAPL'

# Download from Yahoo ?
REFRESH    = False

START_DATE = '2017-07-15'
END_DATE   = '2021-03-22'

### Data download

In [3]:
raw = sc.load_security(dirname = DIRNAME, 
                       ticker  = TICKER, 
                       refresh = REFRESH, 
                       period  = DEFAULT_PERIOD,
                      )

Loading ticker AAPL
[*********************100%***********************]  1 of 1 completed


In [4]:
security = pd.DataFrame(raw[f'Close_{TICKER}'])
security.rename(columns={f'Close_{TICKER}': "Close"},
                inplace=True)

security

,Close
Date,
2016-03-23,24.68
2016-03-24,24.57
2016-03-28,24.46
2016-03-29,25.04
2016-03-30,25.48
...,...
2021-03-17,124.76
2021-03-18,120.53
2021-03-19,119.99


#### Setup default start & end dates

In [5]:
def init_dates():
    start_dt    = datetime.strptime(START_DATE, '%Y-%m-%d')
    end_dt      = datetime.strptime(END_DATE, '%Y-%m-%d')
    secu_start  = security.index[0]
    if secu_start > start_dt:
        start_dt = secu_start
        print(f'start date reset to {start_dt}')
    return start_dt, end_dt

def get_title_dates():
    ''' 
    Return start and end dates for title format:
    %d-%b-%Y: 03-Jul-2021
    '''
    dates = init_dates()
    return dates[0].strftime('%d-%b-%Y'), dates[1].strftime('%d-%b-%Y')

def get_datetime_dates():
    '''
    Return start and end dates in datetime format
    '''
    return init_dates()

def get_filename_dates():
    '''
    Return start and end dates for file name format:
    %Y-%m-%d: 2021-03-07
    '''
    dates = init_dates()
    return dates[0].strftime('%Y-%m-%d'), dates[1].strftime('%Y-%m-%d')
    
print(get_title_dates())
print(get_datetime_dates())
print(get_filename_dates())

('15-Jul-2017', '22-Mar-2021')
(datetime.datetime(2017, 7, 15, 0, 0), datetime.datetime(2021, 3, 22, 0, 0))
('2017-07-15', '2021-03-22')


## Trading strategy

### Supporting routines

In [6]:
def get_fee(data, fee_pct):
    '''
    Return fees associated to position switches
    fee_pct -> brokers fee 
    fee -> $ fee corresponding to fee_pct
    '''
    # Add a fee for each movement
    fee = (fee_pct * data[data.SWITCH == switches[0]].CUMRET_EMA.sum())
    fee += (fee_pct * data[data.SWITCH == switches[1]].CUMRET_EMA.sum())
    return fee

In [7]:
def get_cumret(data, strategy, fee=0):
    '''
    Returns cumulative return for the given strategy
    If strategy is EMA, returns cumulative returns net of fees
    '''
    if strategy.lower() == 'hold':
        return data.CUMRET_HOLD[-1]/initial_wealth - 1
    elif strategy.lower() == 'ema':
        return (data.CUMRET_EMA[-1]-fee)/initial_wealth - 1
    else:
        raise ValueError(
            f"strategy {strategy} should be either of ema or hold")

In [8]:
def display_full_dataframe(data):
    '''
    Displays all rows & columns of a pandas dataframe
    '''
    with pd.option_context('display.max_rows', None,
                           'display.max_columns', None,
                           'display.width', 1000,
                           'display.precision', 2,
                           'display.colheader_justify',
                           'left'):
        display(data)

### Main algorithms

In [9]:
positions = ['long', 'short', 'n/c']
switches = ['buy', 'sell', 'n/c']

def build_positions(df):
    '''
    This agorithm builds desired positions for the EMA strategy 
    *** Long strategy only ***
    POSITION -> cash, long (short pending)
    SWICH -> buy, sell, n/c (no change)
    '''
    n_time_steps = df.shape[0]
    positions, switches = ([] for i in range(2))

    positions.append('cash')
    switches.append('n/c')

    for step in range(1, n_time_steps):  # Long strategy only
        price = df.loc[df.index[step], 'Close']
        mean = df.loc[df.index[step], 'EMA']
        if positions[step - 1] == 'cash':  # previous position: cash
            if df.loc[df.index[step], 'SIGN'] in [0, -1]:
                positions.append('cash')
                switches.append('n/c')
            elif df.loc[df.index[step], 'SIGN'] == 1:
                positions.append('long')
                switches.append('buy')
            else:
                raise ValueError(f'Inconsistent sign')
        elif positions[step - 1] == 'long':  # previous position: long
            if df.loc[df.index[step], 'SIGN'] in [0, 1]:
                positions.append('long')
                switches.append('n/c')
            elif df.loc[df.index[step], 'SIGN'] == -1:
                positions.append('cash')
                switches.append('sell')
            else:
                raise ValueError(f'Inconsistent sign')
        else:
            raise ValueError(
                f"step {step}: previous position {positions[step - 1]} sign:{df.loc[df.index[step], 'SIGN']}")
    df.insert(loc=len(df.columns), column='POSITION', value=positions)
    df.insert(loc=len(df.columns), column='SWITCH', value=switches)
    return df

def get_strategy(df, span, buffer, debug=False, reactivity=1):
    '''
    ***At this point, only a long strategy is considered***
    Implements running-mean (ewm) strategy
    Input dataframe df has date index & security value 'Close'

    Variables:
    span       -> number of rolling days
    reactivity -> reactivity to market change in days (should be set to 1)
    buffer     -> % above ema to trigger buy

    Returns:
    A dataframe with original data + following columns
    EMA -> exponential moving average
    SIGN -> 1 : above buffer 0: in buffer -1: below buffer
    SWITCH -> buy / sell / n/c (no change)
    RET -> 1 + % daily return
    CUMRET_HOLD -> cumulative returns for a hold strategy
    RET2 -> 1 + % daily return when Close > EMA (***long strategy only***)
    CUMRET_EMA -> cumulative returns for the EMA strategy
    '''
    # Compute exponential weighted mean
    df['EMA'] = df.Close.ewm(span=span, adjust=False).mean()
    if debug:  # include buffer limits in dataframe for printing
        df.insert(loc=1, column='EMA-', value=df['EMA']*(1-buffer))
        df.insert(loc=len(df.columns), column='EMA+',
                  value=df['EMA']*(1+buffer))
    df['SIGN'] = np.where(df.Close - df.EMA*(1 + buffer) > 0, 1,
                          np.where(df.Close - df.EMA*(1 - buffer) < 0, -1, 0)
                          )
    df['SIGN'] = df['SIGN'].shift(reactivity)
    df.loc[df.index[0], 'SIGN'] = 0.0  # set first value to 0

    df = build_positions(df)  # compute position and switch columns

    # Compute returns, cumulative returns and 'wealth' for hold strategy
    df['RET'] = 1.0 + df.Close.pct_change()
    df.loc[df.index[0], 'RET'] = 1.0  # set first value to 1.0
    df['CUMRET_HOLD'] = initial_wealth * df.RET.cumprod(axis=None, skipna=True)

    # Cumulative wealth for ema strategy
    df['RET_EMA'] = np.where(df.POSITION == 'long', df.RET, 1.0)
    df['CUMRET_EMA'] = df.RET_EMA.cumprod(
        axis=None, skipna=True) * initial_wealth
    df.loc[df.index[0], 'CUMRET_EMA'] = initial_wealth
    df = df.drop(['RET_EMA'], axis=1)
    df = df.drop(['SIGN'], axis=1)
    df.to_csv('check.csv', sep=',')
    return df


#### Test driver for main algorithm

start_date = '2018-12-31'
end_date = '2021-03-19'
span   = 29
buffer = .015
reactivity = 1
df = get_strategy(security.loc[start_date:end_date, :].copy(),
                  span,
                  buffer)
df

## Plotting functions

#### Default plot parameters

In [10]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
year_month_fmt = mdates.DateFormatter('%b-%y')
title_size     = 14
vline_color    = 'tomato'
gridcolor      = '#ededed'
title_color    = 'dimgrey'
fig_width      = 14
fig_height     = 8

### Supporting routines

In [11]:
def save_figure(p_file, dpi=360, extension='png'):
    '''
    Save figure to file
    '''
    filename = os.path.join(PLOT_DIR, p_file + f'.{extension}')
    plt.savefig(filename,
                dpi=dpi,
                transparent=False,
                orientation='landscape',
                bbox_inches='tight')

In [12]:
def plot_arrows(axis, data):
    ''' 
    Draws position-switching arrows on axis
    '''
    vertical_range = data['Close'].max() - data['Close'].min()
    horizont_range = (data.index[-1] - data.index[0]).days
    arrow_length   = vertical_range/10
    head_width     = horizont_range/100
    head_length    = vertical_range/50
    space          = vertical_range/100  # space bw arrow tip and curve

    for row in range(data.shape[0]):
        if data.SWITCH[row] == switches[0]:  # buy
            y_start = data.loc[data.index[row], 'Close'] + arrow_length
            color = colors[2]
            dy = -arrow_length+space
        elif data.SWITCH[row] == switches[1]:  # sell
            y_start = data.loc[data.index[row], 'Close'] - arrow_length
            color = colors[3]
            dy = arrow_length-space
        else:  # don't draw the arrow
            continue
        arrow_start = data.loc[data.index[row], 'Close']
        axis.arrow(x=data.index[row], y=y_start,
                   dx=0, dy=dy,
                   head_width=head_width,
                   head_length=head_length,
                   length_includes_head=True,
                   linestyle='-',
                   color=color,
                  )

### Main plotting function

In [13]:
def plot_moving(span, fee_pct, buffer, date_range, reactivity=1):
    '''
    Plots price with moving average
    span -> rolling span
    fee -> fee associated with switching position
    reactivity -> time lag in days after Close (normally = 1)
    '''
    title_date_format = '%d-%b-%Y'
    start = date_range[0]
    end   = date_range[1]
    start_string = start.strftime('%d-%b-%Y')
    end_string   = end.strftime('%d-%b-%Y')
    
    #start = datetime.strptime(start_date, "%Y-%m-%d")
    #end = datetime.strptime(end_date, "%Y-%m-%d")

    # Extract time window
    df = get_strategy(security.loc[start:end, :].copy(),
                      span,
                      buffer,
                     )
    fee = get_fee(df, fee_pct)
    hold = get_cumret(df, 'hold')  # cumulative returns for hold strategy
    ema = get_cumret(df, 'ema', fee)  # cumulative returns for EMA strategy

    fig, ax = plt.subplots(figsize=(14, 8))
    ax.plot(df.index, df.Close, linewidth=1, label='Price')
    ax.plot(df.index, df.EMA, linewidth=1, label=f'{span}-days EMA')

    ax.legend(loc='best')
    ax.set_ylabel('Price ($)')
    ax.xaxis.set_major_formatter(year_month_fmt)
    ax.grid(b=None, which='both', axis='both',
             color=gridcolor, linestyle='-', linewidth=1)

    title  = f'{TICKER} | '
    title += f'{start.strftime(title_date_format)} - {end.strftime(title_date_format)}\n'
    title += f'EMA payoff={ema:.2%} (Hold={hold:.1%}) | '
    title += f'{span}-day rolling mean | {buffer:.2%} buffer'
    ax.set_title(title, fontsize=title_size, color=title_color)

    plot_arrows(ax, df)
    save_figure(f'{TICKER}_{start_string}_{end_string}')
    return df

#### Test driver for main plot

In [14]:
#start_date = '2017-07-15'
#df = plot_moving(span=29, fee_pct=.004, buffer=.02)

# display_full_dataframe(df)

In [15]:
def plot_max_values(df, ax, n_values, max_val, min_val, form):
    largest_idx = pd.Series(df['ema'].nlargest(n_values)).index
    for large in largest_idx:
        ax.axvline(df.iloc[large][0],
                   color=vline_color,
                   linestyle=':',
                   linewidth=1)
        if form.lower() == 'integer':
            text = f'{df.iloc[large][0]:.0f}'
        elif form.lower() == 'percent':
            text = f'{df.iloc[large][0]:.2%}'
        else:
            raise ValueError(f'{form} not implemented / shoudl be integer or percent')
            
        ax.annotate(text=text,
                    xy=(df.iloc[large][0],
                        min_val + np.random.uniform(0, 1)*(max_val-min_val)),
                    color=vline_color,
                    )
    return largest_idx

In [16]:
def plot_setup(df, target, xlabel):
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))
    ax.plot(df[target],
            df.ema,
            linewidth=1,
            label='EMA return',
           )
    ax.legend(loc='best')
    ax.set_xlabel(xlabel)
    ax.set_ylabel('return (x)')
    plt.grid(b=None, which='major', axis='both', color='#f1f1f1')
    return fig, ax

In [17]:
# Finds optimal span for a given buffer size
def plot_span_range(buffer, n_values, fee_pct, date_range, extension='png'):
    emas = []
    span_range = [5, 60]
    spans = np.arange(span_range[0],
                      span_range[1] + 1)
    start = date_range[0]
    end   = date_range[1]
    start_string = start.strftime('%d-%b-%Y')
    end_string   = end.strftime('%d-%b-%Y')

    for i, span in enumerate(spans):
        df = get_strategy(security.loc[start:end, :].copy(),
                          span,
                          buffer,)
        fee = get_fee(df, fee_pct)
        ema = get_cumret(df, 'ema', fee)
        emas.append(ema)
        if i == 0:
            hold = get_cumret(df, 'hold')
    df = pd.DataFrame(data=[spans, emas]).T
    df.columns = ['span', 'ema']
    
    max_val = df['ema'].max()
    min_val = df['ema'].min()

    # Plot
    fig, ax     = plot_setup(df, target='span', xlabel='rolling mean span (days)')
    largest_idx = plot_max_values(df, ax, n_values, max_val, min_val, 'integer')

    title  = f'{TICKER} | {start_string} - {end_string}\n'
    title += f'EMA max payoff={max_val:.2%} (hold={hold:.2%}) | '
    title += f'opt span={df.iloc[largest_idx[0]][0]:.0f} days | '
    title += f'{buffer:.2%} buffer'
    ax.set_title(title, fontsize=title_size, color=title_color)

    # save plot to file
    save_figure(f'{TICKER}_{start_string}_{end_string}_spans')

In [18]:
# Finds optimal buffer size for a given rolling-window span

def plot_buffer_range(span, n_values, fee_pct, date_range, extension='png'):
    buffer_range = [0, .03]
    buffers = np.linspace(buffer_range[0],
                          buffer_range[1],
                          101)
    start = date_range[0]
    end   = date_range[1]
    start_string = start.strftime('%d-%b-%Y')
    end_string   = end.strftime('%d-%b-%Y')
    
    emas = []
    for i, buffer in enumerate(buffers):
        df = get_strategy(security.loc[start:end, :].copy(),
                          span,
                          buffer,)
        if i == 0:
            hold = get_cumret(df, 'hold')
        fee = get_fee(df, fee_pct)
        emas.append(get_cumret(df, 'ema', fee))
        
    df = pd.DataFrame(data=[buffers, emas]).T
    df.columns = ['buffer', 'ema']
    
    max_val = df['ema'].max()
    min_val = df['ema'].min()

    # Plot
    fig, ax     = plot_setup(df, target='buffer', xlabel='buffer size (% around EMA)')
    largest_idx = plot_max_values(df, ax, n_values, max_val, min_val, 'percent')

    title  = f'{TICKER} | {start_string} - {end_string}\n'
    title += f'EMA max payoff={max_val:.2%} (hold={hold:.2%}) | '
    title += f'{span}-day mean | '
    title += f'opt buffer={df.iloc[largest_idx[0]][0]:.2%}' 
    ax.set_title(title, fontsize=title_size, color=title_color)
 
    save_figure(f'{TICKER}_{start_string}_{end_string}_buffers')

In [ ]:
# Contour plot of EMA for all window span/buffer size combinations

def plot_maxima(emas, spans, buffers, hold, ax, n_maxima):
        '''
        Plot points corresponding to n_maxima largest EMA  
        '''
        for i in range(n_maxima):
            # Get coordinates of maximum emas value
            max_idx = np.unravel_index(np.argmax(emas, axis=None),
                                       emas.shape)
            if i == 0: # Save best EMA
                max_ema  = np.max(emas) 
                max_span = spans[max_idx[0]]
                max_buff = buffers[max_idx[1]]

            ax.plot(buffers[max_idx[1]], spans[max_idx[0]], marker = 'x')
            ax.annotate(i, xy=(buffers[max_idx[1]],
                               spans[max_idx[0]]), 
                           textcoords='data',
                           fontsize=8,
                        )
            print(f'Max EMA {i}={np.max(emas):.2%}: {spans[max_idx[0]]:.0f}-days buffer={buffers[max_idx[1]]:.2%} (hold={hold:.2%})')
            
            # set max emas value to arbitrily small number and re-iterate
            emas[max_idx[0]][max_idx[1]] = - HUGE
        return max_ema, max_span, max_buff

def plot_buffer_span_contours(fee_pct=FEE_PCT):
    fig_size   = (14, 14)
    n_contours = 10 # number of contours
    n_maxima   = 8 # number of maximum points to plot
    n_buffers  = 101
    
    # Get start & end dates in various fomats
    start, end             = get_datetime_dates()
    start_title, end_title = get_title_dates()
    start_name, end_name   = get_filename_dates()
    
    # define rolling window span range
    span_range = [5, 60]
    spans = np.arange(span_range[0],
                      span_range[1] + 1, 
                      step = 1)
    
    # define buffer range
    buffer_range = [0, .03]
    buffers = np.linspace(buffer_range[0],
                          buffer_range[1],
                          n_buffers)
    # Initialize EMA returns 
    emas = np.zeros((spans.shape[0], buffers.shape[0]), dtype=np.float64)
    
    # Fill EMAS for all span/buffer combinations
    tqdm_desc = f'Outer Level / {span_range[1]-span_range[0]+1}'
    for i, span in tqdm(enumerate(spans), desc=tqdm_desc):
        for j, buffer in enumerate(buffers):
            df  = get_strategy(security.loc[start:end, :].copy(),
                               span,
                               buffer,) 
            emas[i][j] = get_cumret(df, 'ema', get_fee(df, fee_pct))
            if i == 0 and j == 0:
                hold = get_cumret(df, 'hold')

    # Plot
    fig, ax = plt.subplots(figsize=fig_size)
    plt.contourf(buffers, spans, emas, 
                 levels=n_contours, 
                 cmap='viridis', 
                 )
    plt.colorbar(label='EMA return')
            
    # Plot maxima points 
    max_ema, max_span, max_buff = plot_maxima(emas, spans, buffers, hold, ax, n_maxima)
        
    # Axis labels
    ax.set_xlabel('buffer')
    ax.set_ylabel('span')
    
    # Build title
    title  = f'{TICKER} | {start_title} - {end_title}\n'
    title += f'EMA max payoff={max_ema:.2%} (hold={hold:.2%}) | '
    title += f'{max_span:.0f}-day mean | '
    title += f'opt buffer={max_buff:.2%}'
    ax.set_title(title, fontsize=title_size, color=title_color)
    
    plt.grid(b=None, which='major', axis='both', color=gridcolor)
    save_figure(f'{TICKER}_{start_name}_{end_name}_contours')
    plt.show()
    
plot_buffer_span_contours()
    

Outer Level / 56: 0it [00:00, ?it/s]

### Interactive plots

#### Widget defaults

In [ ]:
string_style = {'description_width': 'initial', 
                'handle_color':      'lawngreen'}

#start_period = security.index[0] # beginning of downloaded data
#end_period   = datetime.strptime(END_DATE, "%Y-%m-%d") # set in default parameters
start_period, end_period = get_datetime_dates()
dates   = pd.date_range(start_period, end_period, freq='D')
options = [(date.strftime(' %d/%m/%Y'), date) for date in dates]
index   = (0, len(options)-1)

# Tx/broker's fee
min_fee     = 0.0
max_fee     = .01
delta_fee   = .0005
default_fee = FEE_PCT

# of days for running mean
min_span     = 1
max_span     = 90
delta_span   = 1
default_span = 20

# buffer around EMA
min_buffer     = 0
max_buffer     = .03
delta_buffer   = .0001
default_buffer = .01

# number of maxima
max_value     = 12
default_value = 10

#start_date = '2017-07-15'

#### Interactive returns for window span

In [ ]:
# Mean window span range optimizer interactive

buffer_slider = FloatSlider(min=min_buffer,
                            max=max_buffer,
                            step=delta_buffer,
                            value=default_buffer,
                            style=string_style,
                            continuous_update=False,
                            description='Buffer (%):',
                            disabled=False,
                            readout=True,
                            readout_format='.2%',
                            layout      = {'width': '250px'},
                           )

max_values_slider = SelectionSlider(options=[x for x in range(0, max_value+1)],
                                    value=default_value,
                                    description='Number of maxima:',
                                    style=string_style,
                                    disabled=False,
                                    continuous_update=False,
                                    readout=True,
                                    readout_format='d',
                                    layout      = {'width': '500px'},
                                   )

fee_slider = FloatSlider(min=min_fee,
                         max=max_fee,
                         step=delta_fee,
                         value=default_fee,
                         description='Fees (%):',
                         style=string_style,
                         disabled=False,
                         continuous_update=False,
                         readout=True,
                         readout_format='.2%',
                         layout      = {'width': '250px'},
                        )

date_range_slider = SelectionRangeSlider(options = options,
                                         index   = index,
                                         description = 'Period:',
                                         orientation = 'horizontal',
                                         style       = string_style,
                                         layout      = {'width': '500px'}
                                        )

ui  = widgets.HBox([buffer_slider, fee_slider, max_values_slider])
out = widgets.interactive_output(plot_span_range,
                                 {'buffer': buffer_slider,
                                  'n_values': max_values_slider,
                                  'fee_pct': fee_slider,
                                  }
                                 )

out = interactive_output(plot_span_range,
                         {'buffer'      : buffer_slider,
                          'n_values'  : max_values_slider,
                          'fee_pct'   : fee_slider,
                          'date_range': date_range_slider,
                          }
                         )

ui = TwoByTwoLayout(top_left     = fee_slider,
                    top_right    = date_range_slider,
                    bottom_left  = buffer_slider,
                    bottom_right = max_values_slider,
                    )

display(ui, out)

#### Interactive returns for buffer size

In [ ]:
# Buffer range optimizer interactive 
span_slider = IntSlider(min=min_span,
                        max=max_span,
                        step=delta_span,
                        value=default_span,
                        description='Mean span:',
                        style=string_style,
                        disabled=False,
                        continuous_update=False,
                        readout=True,
                        readout_format='d',
                        layout = {'width': '250px'},
                       )

max_values_slider = SelectionSlider(options=[x for x in range(0, max_value+1)],
                                    value=default_value,
                                    description='Number of maxima:',
                                    style=string_style,
                                    disabled=False,
                                    continuous_update=False,
                                    readout=True,
                                    readout_format='d',
                                    layout = {'width': '500px'},
                                   )

fee_slider = FloatSlider(min=min_fee,
                         max=max_fee,
                         step=delta_fee,
                         value=default_fee,
                         description='Fees (%):',
                         style=string_style,
                         disabled=False,
                         continuous_update=False,
                         readout=True,
                         readout_format='.2%',
                         layout = {'width': '250px'},
                        )

date_range_slider = SelectionRangeSlider(options = options,
                                         index   = index,
                                         description = 'Period:',
                                         orientation = 'horizontal',
                                         style  = string_style,
                                         layout = {'width': '500px'},
                                        )

out = interactive_output(plot_buffer_range,
                         {'span'      : span_slider,
                          'n_values'  : max_values_slider,
                          'fee_pct'   : fee_slider,
                          'date_range': date_range_slider,
                          }
                         )

ui = TwoByTwoLayout(top_left     = fee_slider,
                    top_right    = date_range_slider,
                    bottom_left  = span_slider,
                    bottom_right = max_values_slider,
                    )

display(ui, out)

#### Interactive buffer-span

In [ ]:
# Buffer-span interactive plot

fee_slider = FloatSlider(min=min_fee,
                         max=max_fee,
                         step=delta_fee,
                         value=default_fee,
                         description='Fees (%):',
                         style=string_style,
                         disabled=False,
                         continuous_update=False,
                         readout=True,
                         readout_format='.2%',
                         layout={'width': '250px'},
                         )

span_slider = IntSlider(min=min_span,
                        max=max_span,
                        step=delta_span,
                        value=default_span,
                        description='Mean span:',
                        style=string_style,
                        disabled=False,
                        continuous_update=False,
                        readout=True,
                        readout_format='d',
                        layout={'width': '250px'},
                        )

buffer_slider = FloatSlider(min=min_buffer,
                            max=max_buffer,
                            step=delta_buffer,
                            value=default_buffer,
                            style=string_style,
                            continuous_update=False,
                            description='Buffer (%):',
                            disabled=False,
                            readout=True,
                            readout_format='.2%',
                            layout={'width': '500px'},
                            )

# Date range slider
date_range_slider = SelectionRangeSlider(options = options,
                                         index   = index,
                                         description = 'Period:',
                                         orientation = 'horizontal',
                                         style       = string_style,
                                         layout      = {'width': '500px'}
                                        )


out = interactive_output(plot_moving,
                         {'span'      : span_slider,
                          'buffer'    : buffer_slider,
                          'fee_pct'   : fee_slider,
                          'date_range': date_range_slider,
                          }
                         )

ui = TwoByTwoLayout(top_left     = fee_slider,
                    top_right    = date_range_slider,
                    bottom_left  = span_slider,
                    bottom_right = buffer_slider,
                    )

display(ui, out)